In [ ]:
!pip install pynapple matplotlib dandi dandischema


Grid Cell Tutorial
=========================

*Author: Dhruv Mehrotra and Guillaume Viejo*


This notebook demonstrates how we use Pynapple on various publicly available datasets in systems neuroscience to streamline analysis. In this notebook, we will examine the dataset from [Sargolini et al (2006)](https://www.science.org/doi/10.1126/science.1125572). The entire dataset can be downloaded  <a href="https://dandiarchive.org/dandiset/000582" target="_blank">here</a>. 

***

**Note:** This tutorial uses matplotlib for displaying the figure as well as the dandi package.

You can install all with `pip install matplotlib dandi dandischema`

***

DANDI
-----
DANDI allows you to stream data without downloading all the files. In this case, the data extracted from the NWB file are stored in the nwb-cache folder. To do so, run the following lines of code:



In [ ]:
from pynwb import NWBHDF5IO

from dandi.dandiapi import DandiAPIClient
import fsspec
from fsspec.implementations.cached import CachingFileSystem
import h5py


# Ecephys
dandiset_id, filepath = (
    "000582",
    "sub-10073/sub-10073_ses-17010302_behavior+ecephys.nwb",
)


with DandiAPIClient() as client:
    asset = client.get_dandiset(dandiset_id, "draft").get_asset_by_path(filepath)
    s3_url = asset.get_content_url(follow_redirects=1, strip_query=True)

# First, create a virtual filesystem based on the http protocol
fs = fsspec.filesystem("http")

# Create a cache to save downloaded data to disk (optional)
fs = CachingFileSystem(
    fs=fs,
    cache_storage="nwb-cache",  # Local folder for the cache
)

# Next, open the file
file = h5py.File(fs.open(s3_url, "rb"))
io = NWBHDF5IO(file=file, load_namespaces=True)

Pynapple
--------
If opening the NWB works, you can start streaming data straight into pynapple with the `NWBFile` class.



In [ ]:
#Import matplotlib.pyplot, numpy, pynapple and seaborn libraries

data = nap.NWBFile(io.read())

print(data)

We can load the spikes as a TsGroup for inspection.



In [ ]:
# Get spike timings
spikes = 

# What does this look like? 
print(spikes)

This dataset contains units recorded from layer 2 of the medial entorhinal cortex. This brain region has neurons that fire periodically in space, forming a grid-like pattern in the environment. These cells are called *grid cells*, and their discovery was the basis of the Nobel Prize in Physiology in the year 2014!

To examine the activity of these cells, we plot their firing rate as a function of the animal's position. This plot is referred to as a 'tuning curve'. To plot tuning curves, we need the spike timings and the position information of the animal. 

Let's look at the position of the animal:

In [ ]:
position = data["SpatialSeriesLED1"]

# What does this look like? 
print(position)

Computing tuning curves in Pynapple takes just a single line of code! 

**Note**: Position is a 2-dimensional variable, so use the right tuning curve function! 



In [ ]:
# Compute the tuning curves 

tuning_curves, xybins = 

If you print the variable above, you will discover a 2-dimensional histogram of spikes binned according to the X- and Y- positions, for each unit.

Let's now visualize the tuning curves:



In [ ]:
# Plot the tuning curves 



These units all have periodic firing patterns that span, or *tesselate* the square recording arena. They are typical examples of grid cells.

Now, let's plot the trajectory of the animal, and overlay the spike timings of the second unit (which has a nice *grid field*). This way, we can appreciate the periodic firing patterns of grid cells just by visualizing the raw data!

**Hint**: We need to use a Pynapple function that would help determine the closest position in the time support for each spike.

In [ ]:
#Plot the trajectory of the animal and overlay the spike timings of the second unit



This plot verifies what we visualized in the grid cell tuning curves above, that this cell has a periodic firing pattern that tesselates the recording arena. Therefore, this is a grid cell!

I hope this tutorial was helpful. If you have any questions, comments or suggestions, please feel free to reach out to the Pynapple Team!